# YouTube Comment Cleaning and Exploration

Takes data from a CSV file and cleans the data to isolate comments of interest

In [1]:
import re
import nagisa
import pandas as pd
from collections import Counter
from fugashi import Tagger
from pathlib import Path

NumExpr defaulting to 8 threads.


In [2]:
input_path = Path('Resources/youtube_comments.csv')
comments_df = pd.read_csv(input_path, index_col=0)

comments_df.head(10)

,channel,video_id,category_id,text,date_published,comment_type
0,mwamjapan,Jb6Zlg30rgk,10,This season is going to be a masterpiece <3,2023-04-16T15:39:10Z,top-level
1,mwamjapan,Jb6Zlg30rgk,10,Every season is masterpiece 🔥🔥🔥,2023-04-17T16:41:52Z,reply
2,mwamjapan,Jb6Zlg30rgk,10,@HM cry about it,2023-04-17T16:15:19Z,reply
3,mwamjapan,Jb6Zlg30rgk,10,@HM dude demon slayer has no story but it has ...,2023-04-17T15:44:29Z,reply
4,mwamjapan,Jb6Zlg30rgk,10,@HM 🙂🙃😒😒😒,2023-04-17T13:44:05Z,reply
5,mwamjapan,Jb6Zlg30rgk,10,Honestamente no entiendo porqué esta canción r...,2023-04-17T03:58:25Z,top-level
6,mwamjapan,Jb6Zlg30rgk,10,​@みっくん𓈒𓂂◌𝙼𝙸𝚈𝚄 llora pues,2023-04-17T17:15:35Z,reply
7,mwamjapan,Jb6Zlg30rgk,10,たかがOPで世界観が壊れるアニメじゃないわはげたこ,2023-04-17T16:00:09Z,reply
8,mwamjapan,Jb6Zlg30rgk,10,悩めば、イイ！,2023-04-17T14:53:19Z,reply
9,mwamjapan,Jb6Zlg30rgk,10,@みっくん𓈒𓂂◌𝙼𝙸𝚈𝚄 ニワカほどそう言うよね！,2023-04-17T13:11:13Z,reply


In [3]:
# RE patterns needed for JP text
hiragana = r'\u3041-\u3096'
katakana = r'\u30A1-\u30F6'
kanji = r'\u3006\u4E00-\u9FFF'

# Matches to test for existence of text that uses a Japanese script
jp_text = rf'[{hiragana}{katakana}{kanji}]'

# Accepted characters
accepted_char = rf'[a-zA-Z0-9\u30FC\u3005{hiragana}{katakana}{kanji}]'

In [4]:
# Remove @username from replies
# Done before rest of cleaning to ensure a user using a Japanese script doesn't pass filter
comments_df['text'] = comments_df['text'].str.replace(r'@\S+\s', '', regex=True)

In [5]:
# Filter out comments that don't have any Japanese characters
has_jp_char = comments_df.loc[comments_df['text'].str.contains(jp_text, regex=True)]

has_jp_char.sample(10)

,channel,video_id,category_id,text,date_published,comment_type
93,竹原テレビ,-B7szmyyk2s,24,天心選手のボクシングロードは、彼が勝とうが負けようが自分には楽しみです。彼のキラキラしたスタ...,2023-04-11T00:47:30Z,top-level
2,millennium parade Official YouTube Channel,dYzgZVhiqmU,10,すごすぎる,2023-04-17T11:36:27Z,reply
109,竹原テレビ,-B7szmyyk2s,24,畑山さん、見てみないと分かんないと言いながら、終わった後どや顔で『予想通りよ！』は流石に草,2023-04-09T05:30:27Z,top-level
72,オーイシ加藤のピザラジオ,-7AxDmQ8qns,24,オーイシの義務笑い見てると笑みがこぼれてくる,2023-04-15T17:59:57Z,top-level
163,東海オンエア,Vd0ShQDUops,23,1万円貰う方法のどうがあげてます,2023-04-13T14:07:32Z,reply
11,Kuzuha Channel,NwF8wYFaZCE,20,ここ１ヶ月確実に3人ともすごく成長してて見てて楽しかった。明日頑張れ。,2023-04-14T15:10:39Z,top-level
81,ぱかチューブっ!,pjPXzJedwiA,20,必死に走ってる表情、イイよね！\nレースでは浮わついてない硬派な描写してくれるから、本気のス...,2023-04-16T14:01:44Z,reply
154,東海オンエア,ogfbE7ymLRU,23,ターゲットが しばゆめの時は 絶対面白いから好き😂,2023-04-14T03:34:25Z,top-level
4,Leo the football TV from シュワーボ東京,X5Yz86aBDYs,17,しかも映像なし。\nゴートやぷあたんあたりのアマチュアとは格が違うね,2023-04-16T18:34:36Z,reply
11,ぱかチューブっ!,pjPXzJedwiA,20,1‌万円貰う方法のどうがあげてます,2023-04-17T10:39:09Z,reply


In [6]:
# Remove other unnecessary characters - 
def clean_text(text):
    
    text = re.sub(r'\n', '', text)
    text = re.sub(r'\t', '', text)
    text = re.sub(r'\r', '', text)
    text = re.sub(r'https?://[a-zA-Z0-9.-]*[/?[a-zA-Z0-9.-_]*]*', '', text)
    text = re.sub(r'笑+', '笑', text)
    
    filtered_text = ''
    
    for char in text:
        if re.match(accepted_char, char):
            
            filtered_text += char
            
    return filtered_text

In [7]:
# Clean
cleaned_df = has_jp_char.copy()
cleaned_df['text'] = cleaned_df['text'].apply(clean_text)

cleaned_df.head(10)

,channel,video_id,category_id,text,date_published,comment_type
7,mwamjapan,Jb6Zlg30rgk,10,たかがOPで世界観が壊れるアニメじゃないわはげたこ,2023-04-17T16:00:09Z,reply
8,mwamjapan,Jb6Zlg30rgk,10,悩めばイイ,2023-04-17T14:53:19Z,reply
9,mwamjapan,Jb6Zlg30rgk,10,ニワカほどそう言うよね,2023-04-17T13:11:13Z,reply
18,mwamjapan,Jb6Zlg30rgk,10,珍しくショート動画が出てますね最高すぎる,2023-04-17T10:49:06Z,reply
19,mwamjapan,Jb6Zlg30rgk,10,めっちゃ同意,2023-04-17T08:30:39Z,reply
44,mwamjapan,Jb6Zlg30rgk,10,miletとのコラボには最初びっくりしたけど声の相性が良くて特に重なった時に何とも言えない不...,2023-04-17T00:54:46Z,top-level
45,mwamjapan,Jb6Zlg30rgk,10,全く仰るとおりで作中にはいろんな絆ノ奇跡がありますよね特に刀鍛冶の里編のラストは絆ノ奇跡その...,2023-04-17T13:48:13Z,reply
46,mwamjapan,Jb6Zlg30rgk,10,我が命果てようとも繋いでいこう自分が志半ばで死んでも繋いだ命がきっと果たしてくれると二話で炭...,2023-04-17T13:37:24Z,reply
47,mwamjapan,Jb6Zlg30rgk,10,我が命果てようともには無一郎を思い浮かべました,2023-04-17T13:07:22Z,reply
48,mwamjapan,Jb6Zlg30rgk,10,鬼滅の刃には色んな絆ノ奇跡がありますね,2023-04-17T11:53:14Z,reply


In [8]:
# Check size of DF
cleaned_df.shape

(11297, 6)

In [9]:
# Check for null values
cleaned_df.isna().sum()

channel           0
video_id          0
category_id       0
text              0
date_published    0
comment_type      0
dtype: int64

In [10]:
# Check unique values in each
cleaned_df.nunique()

channel              82
video_id             92
category_id          12
text              10659
date_published    11104
comment_type          2
dtype: int64

In [11]:
# Check amounts of top-level vs. replies for comments
cleaned_df['comment_type'].value_counts()

top-level    8320
reply        2977
Name: comment_type, dtype: int64

In [12]:
# Create instance of fugashi tagger
tagger = Tagger()

In [12]:
# Tokenize text
analysis_df = cleaned_df.copy()
analysis_df['tokens'] = analysis_df['text'].apply(lambda x: nagisa.tagging(x))

analysis_df.head(10)

,channel,video_id,category_id,text,date_published,comment_type,tokens
7,mwamjapan,Jb6Zlg30rgk,10,たかがOPで世界観が壊れるアニメじゃないわはげたこ,2023-04-17T16:00:09Z,reply,たか/名詞 が/助詞 OP/名詞 で/助詞 世界/名詞 観/接尾辞 が/助詞 壊れる/動詞 ...
8,mwamjapan,Jb6Zlg30rgk,10,悩めばイイ,2023-04-17T14:53:19Z,reply,悩め/動詞 ば/助詞 イイ/形容詞
9,mwamjapan,Jb6Zlg30rgk,10,ニワカほどそう言うよね,2023-04-17T13:11:13Z,reply,ニワカ/形状詞 ほど/助詞 そう/副詞 言う/動詞 よ/助詞 ね/助詞
18,mwamjapan,Jb6Zlg30rgk,10,珍しくショート動画が出てますね最高すぎる,2023-04-17T10:49:06Z,reply,珍しく/形容詞 ショート/名詞 動画/名詞 が/助詞 出/動詞 て/助動詞 ます/助動詞 ね...
19,mwamjapan,Jb6Zlg30rgk,10,めっちゃ同意,2023-04-17T08:30:39Z,reply,めっちゃ/副詞 同意/名詞
44,mwamjapan,Jb6Zlg30rgk,10,miletとのコラボには最初びっくりしたけど声の相性が良くて特に重なった時に何とも言えない不...,2023-04-17T00:54:46Z,top-level,milet/名詞 と/助詞 の/助詞 コラボ/名詞 に/助詞 は/助詞 最初/名詞 びっくり...
45,mwamjapan,Jb6Zlg30rgk,10,全く仰るとおりで作中にはいろんな絆ノ奇跡がありますよね特に刀鍛冶の里編のラストは絆ノ奇跡その...,2023-04-17T13:48:13Z,reply,全く/副詞 仰る/動詞 とおり/名詞 で/助詞 作中/名詞 に/助詞 は/助詞 いろんな/連...
46,mwamjapan,Jb6Zlg30rgk,10,我が命果てようとも繋いでいこう自分が志半ばで死んでも繋いだ命がきっと果たしてくれると二話で炭...,2023-04-17T13:37:24Z,reply,我が/連体詞 命/名詞 果てよう/動詞 と/助詞 も/助詞 繋い/動詞 で/助詞 いこう/動...
47,mwamjapan,Jb6Zlg30rgk,10,我が命果てようともには無一郎を思い浮かべました,2023-04-17T13:07:22Z,reply,我が/連体詞 命/名詞 果てよう/動詞 とも/名詞 に/助詞 は/助詞 無一郎/名詞 を/助...
48,mwamjapan,Jb6Zlg30rgk,10,鬼滅の刃には色んな絆ノ奇跡がありますね,2023-04-17T11:53:14Z,reply,鬼滅/名詞 の/助詞 刃/名詞 に/助詞 は/助詞 色んな/連体詞 絆ノ/名詞 奇跡/名詞 ...


In [14]:
# This seems like potential!!
# Let's filter now
analysis_df['filtered_tokens'] = (analysis_df['text']
                                  .apply(lambda x: nagisa.filter(x, filter_postags=['助詞', '助動詞'])))

analysis_df.head(10)

,channel,video_id,category_id,text,date_published,comment_type,tokens,filtered_tokens
7,mwamjapan,Jb6Zlg30rgk,10,たかがOPで世界観が壊れるアニメじゃないわはげたこ,2023-04-17T16:00:09Z,reply,たか/名詞 が/助詞 OP/名詞 で/助詞 世界/名詞 観/接尾辞 が/助詞 壊れる/動詞 ...,たか/名詞 OP/名詞 世界/名詞 観/接尾辞 壊れる/動詞 アニメ/名詞 ない/形容詞 は...
8,mwamjapan,Jb6Zlg30rgk,10,悩めばイイ,2023-04-17T14:53:19Z,reply,悩め/動詞 ば/助詞 イイ/形容詞,悩め/動詞 イイ/形容詞
9,mwamjapan,Jb6Zlg30rgk,10,ニワカほどそう言うよね,2023-04-17T13:11:13Z,reply,ニワカ/形状詞 ほど/助詞 そう/副詞 言う/動詞 よ/助詞 ね/助詞,ニワカ/形状詞 そう/副詞 言う/動詞
18,mwamjapan,Jb6Zlg30rgk,10,珍しくショート動画が出てますね最高すぎる,2023-04-17T10:49:06Z,reply,珍しく/形容詞 ショート/名詞 動画/名詞 が/助詞 出/動詞 て/助動詞 ます/助動詞 ね...,珍しく/形容詞 ショート/名詞 動画/名詞 出/動詞 最高/名詞 すぎる/動詞
19,mwamjapan,Jb6Zlg30rgk,10,めっちゃ同意,2023-04-17T08:30:39Z,reply,めっちゃ/副詞 同意/名詞,めっちゃ/副詞 同意/名詞
44,mwamjapan,Jb6Zlg30rgk,10,miletとのコラボには最初びっくりしたけど声の相性が良くて特に重なった時に何とも言えない不...,2023-04-17T00:54:46Z,top-level,milet/名詞 と/助詞 の/助詞 コラボ/名詞 に/助詞 は/助詞 最初/名詞 びっくり...,milet/名詞 コラボ/名詞 最初/名詞 びっくり/名詞 し/動詞 声/名詞 相性/名詞 ...
45,mwamjapan,Jb6Zlg30rgk,10,全く仰るとおりで作中にはいろんな絆ノ奇跡がありますよね特に刀鍛冶の里編のラストは絆ノ奇跡その...,2023-04-17T13:48:13Z,reply,全く/副詞 仰る/動詞 とおり/名詞 で/助詞 作中/名詞 に/助詞 は/助詞 いろんな/連...,全く/副詞 仰る/動詞 とおり/名詞 作中/名詞 いろんな/連体詞 絆ノ/名詞 奇跡/名詞 ...
46,mwamjapan,Jb6Zlg30rgk,10,我が命果てようとも繋いでいこう自分が志半ばで死んでも繋いだ命がきっと果たしてくれると二話で炭...,2023-04-17T13:37:24Z,reply,我が/連体詞 命/名詞 果てよう/動詞 と/助詞 も/助詞 繋い/動詞 で/助詞 いこう/動...,我が/連体詞 命/名詞 果てよう/動詞 繋い/動詞 いこう/動詞 自分/名詞 志/名詞 半ば...
47,mwamjapan,Jb6Zlg30rgk,10,我が命果てようともには無一郎を思い浮かべました,2023-04-17T13:07:22Z,reply,我が/連体詞 命/名詞 果てよう/動詞 とも/名詞 に/助詞 は/助詞 無一郎/名詞 を/助...,我が/連体詞 命/名詞 果てよう/動詞 とも/名詞 無一郎/名詞 思い浮かべ/動詞
48,mwamjapan,Jb6Zlg30rgk,10,鬼滅の刃には色んな絆ノ奇跡がありますね,2023-04-17T11:53:14Z,reply,鬼滅/名詞 の/助詞 刃/名詞 に/助詞 は/助詞 色んな/連体詞 絆ノ/名詞 奇跡/名詞 ...,鬼滅/名詞 刃/名詞 色んな/連体詞 絆ノ/名詞 奇跡/名詞 あり/動詞


In [16]:
test = analysis_df.iloc[0]['filtered_tokens']
test

In [21]:
test.words

['たか', 'OP', '世界', '観', '壊れる', 'アニメ', 'ない', 'はげ', 'たこ']

In [23]:
type(test.words[0])

str

In [28]:
str(test)

'たか/名詞 OP/名詞 世界/名詞 観/接尾辞 壊れる/動詞 アニメ/名詞 ない/形容詞 はげ/名詞 たこ/名詞'

In [54]:
# Extract filtered_tokens into a list
import operator
from functools import reduce

token_list = analysis_df['filtered_tokens'].apply(lambda x: str(x).split()).tolist()
word_list = reduce(operator.add, token_list)

In [57]:
total_words = len(word_list)
print(total_words, word_list[:10])

163992 ['たか/名詞', 'OP/名詞', '世界/名詞', '観/接尾辞', '壊れる/動詞', 'アニメ/名詞', 'ない/形容詞', 'はげ/名詞', 'たこ/名詞', '悩め/動詞']


In [58]:
# Count and sort by occurrences
counter = Counter(word_list)
most_occurrences = counter.most_common()

most_occurrences

[('し/動詞', 3578),
 ('1/名詞', 2347),
 ('さん/接尾辞', 2149),
 ('0/名詞', 2094),
 ('2/名詞', 1848),
 ('3/名詞', 1492),
 ('お/接頭辞', 1439),
 ('5/名詞', 1255),
 ('4/名詞', 1204),
 ('ちゃん/接尾辞', 1182),
 ('見/動詞', 1096),
 ('この/連体詞', 844),
 ('ありがとう/感動詞', 813),
 ('いい/形容詞', 796),
 ('好き/形状詞', 788),
 ('ない/形容詞', 786),
 ('い/動詞', 784),
 ('最高/名詞', 761),
 ('する/動詞', 736),
 ('すぎる/動詞', 730),
 ('こと/名詞', 728),
 ('ござい/動詞', 650),
 ('なっ/動詞', 638),
 ('w/記号', 632),
 ('本当/名詞', 622),
 ('いる/動詞', 602),
 ('これ/代名詞', 598),
 ('動画/名詞', 584),
 ('ある/動詞', 583),
 ('人/名詞', 557),
 ('さ/動詞', 519),
 ('よう/形状詞', 518),
 ('6/名詞', 514),
 ('人/接尾辞', 511),
 ('そう/形状詞', 509),
 ('いう/動詞', 498),
 ('さ/接尾辞', 492),
 ('7/名詞', 479),
 ('思い/動詞', 471),
 ('曲/名詞', 471),
 ('なる/動詞', 466),
 ('くれ/動詞', 443),
 ('様/接尾辞', 433),
 ('8/名詞', 431),
 ('選手/名詞', 417),
 ('すぎ/動詞', 416),
 ('いつ/代名詞', 405),
 ('日本/名詞', 399),
 ('試合/名詞', 393),
 ('思う/動詞', 391),
 ('9/名詞', 389),
 ('思っ/動詞', 382),
 ('自分/名詞', 381),
 ('嬉しい/形容詞', 379),
 ('時/名詞', 376),
 ('良い/形容詞', 367),
 ('楽しみ/名詞', 365),
 ('き/動詞', 358),
 